In [ ]:
# Librairies
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

# === CONFIGURATION ===
# Tes chemins (identiques à ton script)
path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"
output_filename = "donnees_carte_25ans_mensuel.nc"

print(" Démarrage de la création du NetCDF complet...")

try:
    # 1. Sélection des fichiers
    fichiers = sorted(glob.glob(path_data))
    # On prend 25 années (Supposons 1 fichier par an ou par mois, on adapte)
    # On en prend 420.
    fichiers_35ans = fichiers[:420]
    print(f" Fichiers sélectionnés : {len(fichiers_35ans)} (de {os.path.basename(fichiers_35ans[0])} à {os.path.basename(fichiers_35ans[-1])})")

    print(" Chargement du masque...")
    mask = xr.open_dataset(path_mask)

    # Liste pour stocker les morceaux traités
    datasets_mensuels = []

    print("\n Traitement année par année (Conservation Lat/Lon, Moyenne Mensuelle)...")

    for i, fichier in enumerate(fichiers_35ans):
        print(f"   → Traitement fichier {i+1}/{len(fichiers_35ans)}...", end=" ")

        try:
            # A. Chargement d'un fichier
            ds = xr.open_dataset(fichier, engine='netcdf4')

            # Renommage si besoin (valid_time -> time)
            if 'valid_time' in ds.coords:
                ds = ds.rename({'valid_time': 'time'})

            # B. Masquage (On garde la forme de la France)
            ds_masked = ds.where(mask['mask'] == 1)

            # C. Conversion Celsius
            ds_masked['t2m'] = ds_masked['t2m'] - 273.15

            # D. RÉÉCHANTILLONNAGE MENSUEL (C'est le secret pour ne pas saturer)
            # On garde (Lat, Lon) mais on compresse le temps (1 image par mois)
            ds_monthly = ds_masked.resample(time='1M').mean(skipna=True)

            # On stocke ce morceau
            # .compute() force le calcul maintenant pour libérer la mémoire après
            datasets_mensuels.append(ds_monthly)

            ds.close()
            print("✅")

        except Exception as e_file:
            print(f" Erreur sur {fichier} : {e_file}")

    # 3. FUSION FINALE
    print("\n Fusion de tous les mois en un seul fichier...")
    ds_final = xr.concat(datasets_mensuels, dim='time')

    # Nettoyage final (On garde juste la température)
    ds_final = ds_final[['t2m']]
    ds_final = ds_final.rename({'t2m': 'Temperature_C'})

    # 4. SAUVEGARDE
    print(f" Sauvegarde dans '{output_filename}'...")
    ds_final.to_netcdf(output_filename)

    print("\n TERMINÉ ! Le fichier est prêt.")
    print(f"   Taille estimée : Quelques Mo (Transportable)")
    print(f"   Dimensions : {ds_final.dims} (Time x Latitude x Longitude)")

except Exception as e:
    print(f"\n ERREUR BLOQUANTE : {e}")

 Démarrage de la création du NetCDF complet...
 Fichiers sélectionnés : 420 (de era5_t2m_fr_1950_01.nc à era5_t2m_fr_1984_12.nc)
 Chargement du masque...

 Traitement année par année (Conservation Lat/Lon, Moyenne Mensuelle)...
   → Traitement fichier 1/420... 

c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


✅
   → Traitement fichier 2/420... ✅
   → Traitement fichier 3/420... 

c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


✅
   → Traitement fichier 4/420... ✅
   → Traitement fichier 5/420... 

...

✅
   → Traitement fichier 419/420... 

c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


✅
   → Traitement fichier 420/420... 

c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


✅

 Fusion de tous les mois en un seul fichier...
 Sauvegarde dans 'donnees_carte_25ans_mensuel.nc'...

 TERMINÉ ! Le fichier est prêt.
   Taille estimée : Quelques Mo (Transportable)
   Dimensions : FrozenMappingWarningOnValuesAccess({'time': 420, 'latitude': 43, 'longitude': 62}) (Time x Latitude x Longitude)
